networkx-temporal
---

**[PyPI package](https://pypi.org/p/networkx-temporal/)** | **[Documentation](https://networkx-temporal.readthedocs.io/en/latest)** | **[GitHub repository](https://github.com/nelsonaloysio/networkx-temporal)**


In [ ]:
!pip install -q 'networkx-temporal[ipynb]'   # Includes additional libraries used in this notebook.

In [ ]:
# %load_ext autoreload
# %autoreload 2
import networkx as nx
import networkx_temporal as tx

_____

# Basic operations

The examples below cover the package's basic functionalities, including
how to build a temporal graph, slice it into snapshots, save and load
the resulting objects to disk, and other class methods.

## Build temporal graph

The main class of the package is the [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph)
object, which extends [NetworkX
graphs](https://networkx.org/documentation/stable/reference/classes/index.html)
to handle temporal data. Let's start by creating a simple directed graph
using `time` as attribute key:

In [ ]:
TG = tx.TemporalDiGraph()  # TG = tx.temporal_graph(directed=True, multigraph=False)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

print(TG)

Note that the resulting graph object reports a single time step `t=1`,
as it has not yet been [sliced](#slice-temporal-graph).

> **Note:**
> Multigraphs are particularly useful to represent temporal graphs, as it
> allows to store multiple interactions between the same nodes at
> different time steps within a single graph object. This behavior can be
> set by passing `multigraph=True` when creating the
> [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph) object.

## Slice temporal graph

Let's use the [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) method to split
the temporal graph we created into a number of snapshots:

In [ ]:
TG = TG.slice(attr="time")
print(TG)

Inspecting the resulting object's properties can be achieved using some
familiar methods:

In [ ]:
print(f"t = {len(TG)} time steps\n"
      f"V = {TG.order()} nodes ({TG.temporal_order()} unique, {TG.total_order()} total)\n"
      f"E = {TG.size()} edges ({TG.temporal_size()} unique, {TG.total_size()} total)")

We may also visualize the resulting snapshots using the
[`draw`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.draw) function:

In [ ]:
tx.draw(TG, layout="kamada_kawai", figsize=(8, 2))

Note that [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) by default returns a
snapshot for each unique attribute value passed to it.

> **Hint:**
> Setting `names=True` uses the [`names`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.names)
> property as subplot titles, instead of their indices. By default,
> [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) returns the interval of the
> resulting temporal snapshots as their names.

### Number of snapshots

A new object can be created with a specific number of snapshots by
setting the `bins` parameter:

In [ ]:
TG = TG.slice(attr="time", bins=2)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), names=True)

> **Note:**
> In some cases, [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) may still not be able to split the graph into
> the number of snapshots specified by `bins` (e.g., due to insufficient
> data), in which case the maximum possible number is returned instead.
> See [Quantile-based cut](#quantile-based-cut) and [Rank-based
> cut](#rank-based-cut) examples below for alternatives.

### Quantile-based cut

By default, created bins are composed of non-overlapping edges and might
have uneven order and/or size. To try and balance them using quantiles,
pass `qcut=True` (see
[pandas.qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html)
for details):

In [ ]:
TG = TG.slice(attr="time", bins=2, qcut=True)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), names=True)

Though not perfectly balanced due to node $a$ originally appearing
multiple times (in $t=\{0,2,3\}$), the resulting snapshots have more
comparable order and size, with edges still sorted by their `attr`.

This method is particularly useful when node interactions are not evenly
distributed across time. Note that results are highly data-dependent and
are expected to vary in a case-by-case basis.

### Rank-based cut

Forcing a number of bins can be achieved by setting `rank_first=True`,
which sorts edges, nodes, or one of its attributes by their order of
appearance in the data (see
[pandas.Series.rank](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.rank.html)
for details):

In [ ]:
TG = TG.slice(bins=2, rank_first=True)
tx.draw(TG, layout="kamada_kawai", figsize=(4, 2), names=True)

Notice how each snapshot title now refer to edge intervals: $e_0$ to
$e_3$ $(0, 4]$ and $e_4$ to $e_7$ $(4, 8]$. In this case, the graph was
split by sorting the edges by the order in which they were added to the
graph.

Although the resulting number of nodes per snapshot may vary, this
method is useful to obtain an arbitrarily defined number of subgraphs,
especially when its temporal values are not known.

## Import static graphs

Static graph objects may also carry temporal information as node- and
edge-level attributes:

In [ ]:
G = nx.DiGraph()

G.add_nodes_from([
    ("a", {"time": 0}),
    ("b", {"time": 0}),
    ("c", {"time": 1}),
    ("d", {"time": 2}),
    ("e", {"time": 3}),
    ("f", {"time": 3}),
])

G.add_edges_from([
    ("a", "b", {"time": 0}),
    ("c", "b", {"time": 1}),
    ("d", "c", {"time": 2}),
    ("d", "e", {"time": 2}),
    ("a", "c", {"time": 2}),
    ("f", "e", {"time": 3}),
    ("f", "a", {"time": 3}),
    ("f", "b", {"time": 3}),
])

print(G)

We may convert a static graph to a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph)
object using the [`from_static`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.from_static) function:

In [ ]:
TG = tx.from_static(G)
print(TG)

As expected, the resulting object has the same number of nodes and edges
as the original graph. Drawing it with edge labels allows to visualize
the edge-level temporal information in a single plot:

In [ ]:
tx.draw(TG, layout="kamada_kawai", edge_labels="time", suptitle="Temporal Graph (t=1)")

However, note that in the example above, both the nodes and edges are
attributed with a `time` key. Let's see how this affects the resulting
temporal graph when slicing it into snapshots next.

> **See also:**
> The [`from_snapshots`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.from_snapshots) function to import a list of
> static graphs as temporal graph snapshots.

### Edge-level time attribute

Converting a static graph considering edge-level temporal data in to a
temporal graph object:

In [ ]:
TG_ = TG.slice(attr="time")
tx.draw(TG_, layout="kamada_kawai", figsize=(8, 2))

The resulting temporal graph has the same number of edges as the
original graph, but a higher number of nodes. This is expected, as the
same nodes appear in more than one snapshot.

> **Note:**
> By default, [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) as
> an edge-level attribute, which is usually the case for temporal data.
> This behavior can be changed by setting `level='node'`, as seen below.

### Node-level time attribute

Converting a static graph considering node-level temporal data to a
temporal graph object:

In [ ]:
TG_ = TG.slice(attr="time", level="node")
tx.draw(TG_, layout="kamada_kawai", figsize=(8, 2))

Note that now, even though the edge $(a, c)$ contains the attribute
`time=2`, considering node-level attributes resulted in it being
placed at the snapshot $t=0$ instead, as node $a$ is set to `time=0`:

In [ ]:
G.nodes(data="time")["a"]

> **Note:**
> When `level='node'`, the source node's temporal attribute is used by
> default to determine the time step of an edge. This behavior can be
> changed by setting `node_level='target'` instead.

## Save and load data

Temporal graphs may be read from or written to a file using the
following functions:

In [ ]:
tx.write_graph(TG, "temporal-graph.graphml.zip")
TG = tx.read_graph("temporal-graph.graphml.zip")

Supported formats will be automatically detected based on the file
extension. For details on both, please refer to their respective
documentations: [`read_graph`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.read_graph) and
[`write_graph`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.write_graph).

> **See also:**
> The [read and write
> documentation](https://networkx.org/documentation/stable/reference/readwrite/index.html)
> from NetworkX for a list of supported graph formats.

## Other inherited methods

The methods available from a [NetworkX
graph](https://networkx.org/documentation/stable/reference/classes/graph.html#networkx.Graph)
can be called directly from a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph) object
as well. For example, the familiar methods below transform its edges
into directed or undirected:

In [ ]:
TG.to_undirected()

In [ ]:
TG.to_directed()

Note that both methods return new objects when called, so the original
graph remains unchanged.

> **See also:**
> -   The [Appendix → Index](genindex.html) page for a list of the implemented classes, methods, and functions.
> -   The [NetworkX
>     documentation](https://networkx.org/documentation/stable/reference/classes/graph.html#networkx.Graph)
>     for a list of inherited methods found in a [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph).

___

# Common metrics

Both static and temporal graph metrics can be calculated from a
[`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph) object.This section showcases a few common examples
using some of NetworkX's built-in functions and algorithms.

## Static graph metrics

The functions and algorithms implemented by NetworkX can be applied
directly on the temporal graph by iterating over snapshots. For instance,
to calculate the [Katz centrality](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.katz_centrality.html) for each snapshot:

In [ ]:
import networkx as nx
import networkx_temporal as tx

TG = tx.TemporalDiGraph()  # TG = tx.temporal_graph(directed=True, multigraph=False)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

TG = TG.slice(attr="time")

for t, G in enumerate(TG):
    katz = nx.katz_centrality(G)
    katz = {node: round(value, 3) for node, value in katz.items()}
    print(f"t={t}: {katz}")

In addition, any NetworkX [Graph methods](https://networkx.org/documentation/stable/reference/classes/graph.html#methods)
can be called directly from the temporal graph as well.
Doing so will automatically apply it to each snapshot
separately, as seen in the examples below.

> **See also:**
> The [Algorithms section](https://networkx.org/documentation/stable/reference/algorithms/index.html)
> of the NetworkX documentation for a list of available functions.

### Degree centrality

Static graph methods such as
[degree](https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.degree.html),
[in_degree](https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.in_degree.html),
and
[out_degree](https://networkx.org/documentation/stable/reference/classes/generated/networkx.DiGraph.out_degree.html)
return the results per snapshot:

In [ ]:
TG = tx.TemporalDiGraph()  # TG = tx.temporal_graph(directed=True, multigraph=False)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

TG = TG.slice(attr="time")

TG.degree()

Alternatively, we may also obtain the degree of a specific node in a given
snapshot, e.g., node $a_0$:

In [ ]:
TG[0].degree("a")

### Order and size

Similarly, to obtain the total number of nodes and edges in each
snapshot:

In [ ]:
print("Order:", TG.order())
print("Size:", TG.size())

### Node neighborhoods

The [neighbors](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.neighbors.html)
method returns a list of neighbors for each node in each snapshot:

In [ ]:
TG.neighbors("c")

Converting the graph to undirected, we also obtain nodes that have node
$c$ as their neighbor:

In [ ]:
TG.to_undirected().neighbors("c")

## Temporal graph metrics

Only a few methods that consider all snapshots are currently available
from [`TemporalGraph`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph) objects. They mostly serve as
wrappers of methods implemented by NetworkX, for convenience
purposes.

### Temporal degree centrality

The [`temporal_degree`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_degree) method returns a
dictionary containing node degrees across all time steps:

In [ ]:
TG.temporal_degree()

Alternatively, to obtain the degree of a specific node considering all
snapshots, e.g., node $a$:

In [ ]:
TG.temporal_degree("a")

### Temporal order and size

The [`temporal_order`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_order) and
[`temporal_size`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_size) functions return the
total unique nodes and edges:

In [ ]:
print("Temporal nodes:", TG.temporal_order())
print("Temporal edges:", TG.temporal_size())

> **Note:**
> The temporal order and size of a temporal graph match the length of
> [`temporal_nodes`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_nodes) and
> [`temporal_edges`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_edges), i.e., the sets of all
> (**unique**) nodes and edges considering all snapshots.

Alternatively, [`total_order`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.total_order) and
[`total_size`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.total_size) return the sum of nodes
and edges per snapshot:

In [ ]:
print("Total nodes:", TG.total_order())  # TG.total_order() != TG.temporal_order()
print("Total edges:", TG.total_size())   # TG.total_size()  == TG.temporal_size()

> **Note:**
> The total order and size of a temporal graph match the sum of lenghts of
> `nodes` and `edges`, i.e., the lists of all (**non-unique**) nodes and
> edges considering each snapshot separately.

### Temporal node neighborhoods

The [`temporal_neighbors`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.temporal_neighbors) method returns
a dictionary containing node neighbors in all snapshots:

In [ ]:
TG.temporal_neighbors("c")

Converting the graph to undirected, we also obtain nodes that have node
$c$ as their neighbor:

In [ ]:
TG.to_undirected().temporal_neighbors("c")

___

# Convert and transform

The package provides a set of functions to convert to different graph
formats and representations. In this context, ''converting'' refers to
changing the underlying graph object type, e.g.
[igraph](https://igraph.org), while ''transforming'' refers to changing
the graph representation, e.g., [event-based temporal
graphs](#event-based-temporal-graph).

## Graph formats

Graphs may be converted to a different object type by calling
[`convert`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.convert) with the desired format:

In [ ]:
import networkx_temporal as tx

TG = tx.TemporalDiGraph()  # TG = tx.temporal_graph(directed=True, multigraph=False)

TG.add_edge("a", "b", time=0)
TG.add_edge("c", "b", time=1)
TG.add_edge("d", "c", time=2)
TG.add_edge("d", "e", time=2)
TG.add_edge("a", "c", time=2)
TG.add_edge("f", "e", time=3)
TG.add_edge("f", "a", time=3)
TG.add_edge("f", "b", time=3)

TG = TG.slice(attr="time")

tx.convert(TG.to_static(), "igraph")

In the example above, the temporal graph `TG` was first transformed into
a [static graph](#static-graph), which is not required. Otherwise, the
function returns a list of graph objects, one per snapshot, as shown
below:

In [ ]:
tx.convert(TG, "igraph")

Support for the following output formats are implemented, here listed
with their respective aliases:

| Format | Parameter (Package) | Parameter (Alias) |
|---|:---:|:---:|
| [Deep Graph Library](https://www.dgl.ai) | `dgl` | - |
| [DyNetX](https://dynetx.readthedocs.io) | `dynetx` | `dn` |
| [graph-tool](https://graph-tool.skewed.de) | `graph_tool` | `gt` |
| [igraph](https://igraph.org/python) | `igraph` | `ig` |
| [NetworKit](https://networkit.github.io) | `networkit` |``k`` |
| [PyTorch Geometric](https://pytorch-geometric.readthedocs.io) | `torch_geometric` | `pyg` |
| [Teneto](https://teneto.readthedocs.io) | `teneto` | - |

## Graph representations

Once a temporal graph is instantiated, the following methods allow
returning static graphs, snapshots events or unified representations.
Due to the way the underlying data is represented, some of these objects
(i.e., those with unique nodes) do not allow dynamic node attributes.

Observe that the total number of nodes $V$ and edges $E$ of the returned
object might differ from the number of temporal nodes $V_T$ and edges
$E_T$, depending on the data and method used:

| Representation | Oder | Sze  | Dynamic node attributes | Dynamic edge attributes |
| --- | :---: | :---: | :---: | :---: |
| [Static](#static-graph) | $V = V_T$ | $E = E_T$ | ❌ | ✅ |
| [Snapshots](#snapshot-based-temporal-graph) | $V \ge V_T$ | $E = E_T$ | ✅ | ✅ |
| [Events](#event-based-temporal-graph) | $V = V_T$ | $E = E_T$ | ❌ | ❌ |
| [Unified](#unified-temporal-graph) | $V \ge V_T$ | $E \ge E_T$ | ✅ | ✅ |

### Static graph

A static graph `G` is a single graph object containing all the nodes and
edges found in the temporal graph. It is the simplest representation of
a network and is the most common type of graph.

> **Attention:**
> Dynamic node attributes are not preserved when transforming a temporal
> to a static graph.

> **See also:**:
> The [Basic operations → Import static
> graphs](https://networkx-temporal.readthedocs.io/en/latest/examples/basics.html#import-static-graphs) page for more static graph
> conversion examples.

#### `TG` → `G`

Transforming a temporal graph into a static graph with the
[`to_static`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_static) method:

In [ ]:
G = TG.to_static()
print(G)

In [ ]:
tx.draw(G, layout="kamada_kawai", suptitle="Static Graph")

#### `G` → `TG`

Transforming a static graph into a temporal graph with the
[`from_static`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.from_static) function:

In [ ]:
TG = tx.from_static(G)
TG = TG.slice(attr="time")
print(TG)

### Snapshot-based temporal graph

A snapshot-based temporal graph `STG` is a sequence of graphs where each
element represents a snapshot of the original temporal graph. It is the
most common representation of temporal graphs.

> **Note:**
> Like the [`slice`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.slice) method,
> [`to_snapshots`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_snapshots) internally returns views
> of the original graph data, so no data is copied unless specified
> otherwise, i.e., by passing `as_view=False` to the function.

#### `TG` → `STG`

Transforming a temporal graph into a snapshot-based temporal graph with
[`to_snapshots`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_snapshots):

In [ ]:
STG = TG.to_snapshots()
STG

#### `STG` → `TG`

Transforming a snapshot-based temporal graph into a temporal graph with
[`from_snapshots`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.from_snapshots):

In [ ]:
TG = tx.from_snapshots(STG)
print(TG)

### Event-based temporal graph

An event-based temporal graph `ETG` is a sequence of 3- or 4-tuple
edge-based events.

-   **3-tuples** ($u, v, t$), where elements are the source node, target
    node, and time attribute;
-   **4-tuples** ($u, v, t, \varepsilon$), where an additional element
    $\varepsilon$ is either a positive (`1`) or negative (`-1`) unity
    representing edge addition and deletion events, respectively.

Depending on the temporal graph data, one of these may allow a more
compact representation than the other. The default is to return a
3-tuple sequence (also known as a **stream graph**).

> **Important**:
> Event-based temporal graphs do not currently store node- or edge-level
> attribute data. Moreover, as sequences of events are edge-based, node
> isolates are not preserved.

#### `TG` → `ETG`

Transforming a temporal graph into event-based temporal graphs with
[`to_events`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_events):

In [ ]:
ETG = TG.to_events()
ETG

In [ ]:
ETG = TG.to_events(stream=False)
ETG

#### `ETG` → `TG`

Transforming an event-based temporal graph into a temporal graph with
[`from_events`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.from_events):

In [ ]:
TG = tx.from_events(ETG, directed=True, multigraph=True)
print(TG)

### Unified temporal graph

A unified temporal graph `UTG` is a single graph object that contains
the original temporal data, plus ''proxy'' nodes (from each snapshot)
and edge ''couplings'' (linking sequential temporal nodes). Its
usefulness is restricted to certain types of analysis and visualization,
e.g., based on temporal flows.

#### `TG` → `UTG`

Transforming a temporal graph into a unified temporal graph with
[`to_unified`](https://networkx-temporal.readthedocs.io/en/latest/api/classes.html#networkx_temporal.TemporalGraph.to_unified):

In [ ]:
UTG = TG.to_unified(add_couplings=True)
print(UTG)

Let's draw the unified temporal graph to visualize the proxy nodes and
edge couplings (in blue):

In [ ]:
c0, c1 = "#cccccc", "#3987bd"

nodes = sorted(TG.temporal_nodes())

pos = {n: (nodes.index(n.rsplit("_")[0]), -int(n.rsplit("_")[1]))
       for n in UTG.nodes()}

labels = {n: f"{n.split('_')[0]}$_{n.split('_')[1]}$"
          for n in UTG.nodes()}

node_color = [c1 if int(n.split("_")[1]) != TG.index_node(n.split("_")[0])[0] else c0
              for n in UTG.nodes()]

edge_color = [c1 if u.split("_")[0] == v.split("_")[0] else c0
              for u, v in UTG.edges()]

tx.draw(UTG,
        pos=pos,
        figsize=(4, 4),
        labels=labels,
        node_color=node_color,
        edge_color=edge_color,
        connectionstyle="arc3,rad=0.25",
        suptitle="Unified Temporal Graph")

#### `UTG` → `TG`

Transforming a unified temporal graph into a temporal graph with
[`from_unified`](https://networkx-temporal.readthedocs.io/en/latest/api/functions.html#networkx_temporal.from_unified):

In [ ]:
TG = tx.from_unified(UTG)
print(TG)

_____

# Community detection

Community detection is a fundamental task in network analysis. This
simple example demonstrates how a network's temporal dynamics can
overall benefit the detection of its mesoscale structures.

> **Note:**
> Contributions are welcome! If you would like to see a specific algorithm
> for temporal graphs implemented, please feel free to submit a pull
> request on the package's [GitHub
> repository](https://github.com/nelsonaloysio/networkx-temporal).

## Generate graph

As a toy example, let's first use the simplest [Stochastic Block
Model](https://networkx.org/documentation/stable/reference/generated/networkx.generators.community.stochastic_block_model.html)
to generate 4 graph snapshots, in which each of the 5 clusters of 5
nodes each continuously mix together over time:

In [ ]:
snapshots = 4   # Temporal graphs to generate.
clusters = 5    # Number of clusters/communities.
order = 5       # Nodes in each cluster.
intra = .9      # High initial probability of intra-community edges.
inter = .1      # Low initial probability of inter-community edges.
change = .125   # Change in intra- and inter-community edges over time.

# Get probability matrix for each snapshot.
probs = [[[
    (intra if i == j else inter) + (t * change * (-1 if i == j else 1))
    for j in range(clusters)]
    for i in range(clusters)]
    for t in range(snapshots)]

# Create graphs from probabilities.
graphs = {}
for t in range(snapshots):
    graphs[t] = nx.stochastic_block_model(clusters*[order], probs[t], seed=10)
    graphs[t].name = t

# Create temporal graph from snapshots.
TG = tx.from_snapshots(graphs)

Let's plot the graphs, with node colors representing communities and
intra-community edges:

In [ ]:
import matplotlib.pyplot as plt

def get_edge_color(edges: list, node_color: dict):
    return [node_color[u]
            if node_color[u] == node_color[v]
            else "#00000035"
            for u, v in edges]

c = plt.cm.tab10.colors

# Node positions.
pos = nx.circular_layout(TG.to_static())

# Community ground truths.
node_color = [c[i // clusters] for i in range(TG.temporal_order())]

# Colorize intra-community edges.
temporal_opts = {t: {"edge_color": get_edge_color(TG[t].edges(), node_color)}
                 for t in range(len(TG))}

# Plot snapshots with community ground truths.
tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    node_color=node_color,
    temporal_opts=temporal_opts,
    connectionstyle="arc3,rad=0.1",
    suptitle="Ground truths")

We see the graphs are generated with the same community structure, but
continuously decreasing assortativity. Let's try and retrieve the ground
truths using a simple community detection algorithm.

## Modularity optimization

The [leidenalg](https://leidenalg.readthedocs.io) [1] package implements
optimization algorithms for community detection that may be applied on
snapshot-based temporal graphs, allowing to better capture their
underlying structure.

> **Attention:**
> Optimizations algorithms may help with descriptive or exploratory
> tasks and post-hoc network analysis, but lack statistical rigor for
> inferential purposes. See [Peixoto
> (2021)](https://skewed.de/tiago/posts/descriptive-inferential/) [2] for
> a discussion.


### On the static graph

Let's start by considering the network as a ''flattened'' graph, i.e.,
ignoring its temporal information.

We can observe that depending on the initial node community assigments
(e.g., with `seed=0` below),
[modularity](https://leidenalg.readthedocs.io/en/stable/reference.html#modularityvertexpartition) [3]
fails to retrieve the true communities (ground truths) in the network:

In [ ]:
import leidenalg as la

membership = la.find_partition(
    TG.to_static("igraph"),
    la.ModularityVertexPartition,
    n_iterations=-1,
    seed=0,
)

node_color = [c[m] for m in membership.membership]
edge_color = get_edge_color(TG.to_static().edges(), node_color)

tx.draw(
    TG.to_static(),
    pos=pos,
    figsize=(4, 4),
    node_color=node_color,
    edge_color=edge_color,
    connectionstyle="arc3,rad=0.1",
    suptitle="Communities found on static graph")

Next, let's try considering the network's temporal information to see if
we can improve the results.

### On each snapshot

Running the same algorithm separately on each of the generated snapshots
retrieves the correct clusters only on the first graph ($t=0$). In
addition, community indices (represented by their colors) are not fixed
over snapshots, which makes understanding their mesoscale dynamics
harder:

In [ ]:
temporal_opts = {}

for t in range(len(TG)):
    membership = la.find_partition(
        TG[t:t+1].to_static("igraph"),
        la.ModularityVertexPartition,
        n_iterations=-1,
        seed=0,
    )
    node_color = [c[m] for m in membership.membership]
    edge_color = get_edge_color(TG[t].edges(), node_color)
    temporal_opts[t] = {"node_color": node_color, "edge_color": edge_color}

tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    temporal_opts=temporal_opts,
    connectionstyle="arc3,rad=0.1",
    suptitle="Communities found on snapshots")

This is partly due to modularity optimization expecting an assortative
community structure, while the network grew more disassortative over
time. Not only the results of later snapshots are here suboptimal, but
the changing community indices also increases the complexity of their
analysis.

### On the temporal graph

[Coupling temporal
nodes](https://leidenalg.readthedocs.io/en/stable/multiplex.html#slices-to-layers)
allows the same algorithm to correctly retrieve the ground truths in
this case, while at the same time maintaining community indices
consistent over time, as seen below:

In [ ]:
temporal_opts = {}

temporal_membership, improvement = la.find_partition_temporal(
    TG.to_snapshots("igraph"),
    la.ModularityVertexPartition,
    interslice_weight=1.0,
    n_iterations=-1,
    seed=0,
    vertex_id_attr="_nx_name"
)

for t in range(len(TG)):
    node_color = [c[m] for m in temporal_membership[t]]
    edge_color = get_edge_color(TG[t].edges(), node_color)
    temporal_opts[t] = {"node_color": node_color, "edge_color": edge_color}

tx.draw(
    TG,
    pos=pos,
    figsize=(12, 3.5),
    temporal_opts=temporal_opts,
    connectionstyle="arc3,rad=0.1",
    suptitle="Communities found on temporal graph")

As observed, considering the network's temporal information allowed the
algorithm to correctly retrieve the ground truths in this particular
example, as well as to maintain the same community indices across
snapshots, which helps in the study of the network's mesoscale temporal
dynamics.

Although very simple, this example showcases how considering a network's
temporal information can benefit its analysis, as well as help to better
understand and visualize its mesoscale structures, for which this
package provides a flexible framework to work with, especially in a
topological level.

___

**References**

1. V. A. Traag, L. Waltman, N. J. van Eck (2019). ''From Louvain to
   Leiden: guaranteeing well-connected communities''. Scientific Reports,
   9(1), 5233.

2. Tiago. P. Peixoto (2023). ''Descriptive Vs. Inferential Community
   Detection in Networks: Pitfalls, Myths and Half-Truths''. Elements in
   the Structure and Dynamics of Complex Networks, Cambridge University
   Press.

3. Mark Newman (2018). ''Networks''. Oxford University Press, 2nd ed.,
   pp. 498--514.